## Testing Esp32-cam Ip mode - Sending control over the Internet


In [ ]:
import cv2
import socket
import time
import numpy as np
import mediapipe as mp

# =============================================================================
# 1. CONFIGURAÇÕES (VALORES ORIGINAIS DO WAGNER)
# =============================================================================
IP_ESP = "192.168.0.214"
PORTA_UDP = 8888
URL_STREAM = f"http://{IP_ESP}:81/stream"

GANHO_VELOCIDADE = 0.04 # Reduzi levemente para evitar o "coice" inicial
VELOCIDADE_MAX_PASSO = 3.5 # Limita o deslocamento por quadro
DEADZONE = 20 
INTERVALO_COMANDOS = 0.06 

HOME_X, HOME_Y = 100, 45
SERVO_MIN_X, SERVO_MAX_X = 0, 180
SERVO_MIN_Y, SERVO_MAX_Y = 0, 180

# =============================================================================
# 2. INICIALIZAÇÃO
# =============================================================================
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(max_num_hands=1, model_complexity=0, min_detection_confidence=0.7)
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.settimeout(0.02)

# =============================================================================
# 3. ESTADOS E TIMERS
# =============================================================================
STATE_MENU = "MENU"
STATE_CONFIRMACAO = "CONFIRMAR"
STATE_MISSION = "ATIVO"

estado_atual = STATE_MENU
selecao_pendente = None
pos_x, pos_y = float(HOME_X), float(HOME_Y)
ultimo_envio = 0
timer_gesto = 0

# =============================================================================
# 4. FUNÇÕES DE APOIO
# =============================================================================

def analisar_dedos(hl):
    dedos = []
    # Polegar
    if abs(hl.landmark[4].x - hl.landmark[0].x) > abs(hl.landmark[3].x - hl.landmark[0].x):
        dedos.append(1)
    else: dedos.append(0)
    # Indicador, Médio, Anelar, Mínimo
    for p, b in zip([8, 12, 16, 20], [6, 10, 14, 18]):
        dedos.append(1 if hl.landmark[p].y < hl.landmark[b].y else 0)
    return sum(dedos)

def mover_servo_corrigido(tx, ty, cx, cy):
    """Lógica corrigida para evitar que o robô 'fuja'"""
    global pos_x, pos_y, ultimo_envio
    agora = time.time()
    
    if (agora - ultimo_envio) < INTERVALO_COMANDOS:
        return

    # Cálculo do Erro
    erro_x = tx - cx
    erro_y = ty - cy

    # Se o erro for positivo (alvo na direita), precisamos diminuir o ângulo do servo 
    # (ou aumentar, depende da montagem física). 
    # Se ele está fugindo, inverta o sinal de '+' para '-' abaixo:
    if abs(erro_x) > DEADZONE:
        passo_x = np.clip(abs(erro_x) * GANHO_VELOCIDADE, 0, VELOCIDADE_MAX_PASSO)
        pos_x -= passo_x if erro_x < 0 else -passo_x  # <--- INVERTA AQUI SE CONTINUAR FUGINDO

    if abs(erro_y) > DEADZONE:
        passo_y = np.clip(abs(erro_y) * GANHO_VELOCIDADE, 0, VELOCIDADE_MAX_PASSO)
        pos_y -= passo_y if erro_y < 0 else -passo_y

    pos_x = np.clip(pos_x, SERVO_MIN_X, SERVO_MAX_X)
    pos_y = np.clip(pos_y, SERVO_MIN_Y, SERVO_MAX_Y)

    try:
        sock.sendto(f"{int(pos_x)},{int(pos_y)}".encode(), (IP_ESP, PORTA_UDP))
        ultimo_envio = agora
    except: pass

# =============================================================================
# 5. LOOP PRINCIPAL
# =============================================================================
cap = cv2.VideoCapture(URL_STREAM)

while True:
    ret, frame = cap.read()
    if not ret: continue

    frame = cv2.flip(frame, 1)
    ih, iw = frame.shape[:2]
    cx, cy = iw // 2, ih // 2
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    res_hands = hands.process(rgb)
    res_face = face_mesh.process(rgb)
    
    qtd_dedos = -1
    if res_hands.multi_hand_landmarks:
        hl = res_hands.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(frame, hl, mp_hands.HAND_CONNECTIONS)
        qtd_dedos = analisar_dedos(hl)

    # --- LÓGICA DE MENU (1 ou 2 -> Seleciona | Punho 0 -> Confirma) ---
    
    if estado_atual == STATE_MENU:
        cv2.putText(frame, "MOSTRA 1 (MAO) OU 2 (ROSTO)", (50, 50), 1, 2, (0, 255, 255), 2)
        if qtd_dedos in [1, 2]:
            if timer_gesto == 0: timer_gesto = time.time()
            elif time.time() - timer_gesto > 2.0:
                selecao_pendente = qtd_dedos
                estado_atual = STATE_CONFIRMACAO
                timer_gesto = 0
        else: timer_gesto = 0

    elif estado_atual == STATE_CONFIRMACAO:
        modo_txt = "MAO" if selecao_pendente == 1 else "ROSTO"
        cv2.putText(frame, f"CONFIRMAR {modo_txt}? FECHE O PUNHO", (50, 50), 1, 2, (0, 165, 255), 2)
        if qtd_dedos == 0: # Punho fechado
            if timer_gesto == 0: timer_gesto = time.time()
            elif time.time() - timer_gesto > 2.0:
                estado_atual = STATE_MISSION
                timer_gesto = 0
        elif qtd_dedos == 5: # Palma aberta cancela
            estado_atual = STATE_MENU
        else: timer_gesto = 0

    elif estado_atual == STATE_MISSION:
        tx, ty = -1, -1
        if selecao_pendente == 2: # MODO FACE
            if res_face.multi_face_landmarks:
                face_lms = res_face.multi_face_landmarks[0]
                # MÁSCARA TESSELATION (A MAIS BONITA)
                mp_drawing.draw_landmarks(frame, face_lms, mp_face_mesh.FACEMESH_TESSELATION,
                                          landmark_drawing_spec=None,
                                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1))
                p = face_lms.landmark[4] # Nariz
                tx, ty = int(p.x * iw), int(p.y * ih)
        
        elif selecao_pendente == 1: # MODO MAO
            if res_hands.multi_hand_landmarks:
                p = res_hands.multi_hand_landmarks[0].landmark[8] # Indicador
                tx, ty = int(p.x * iw), int(p.y * ih)

        if tx != -1:
            mover_servo_corrigido(tx, ty, cx, cy)
            cv2.circle(frame, (tx, ty), 10, (0, 0, 255), -1)
        
        if qtd_dedos == 5: # Volta ao menu
            estado_atual = STATE_MENU

    cv2.imshow("Tagarela Pro System", frame)
    if cv2.waitKey(1) == ord('q'): break

cap.release()
cv2.destroyAllWindows()